# This notebook demonstrates simple visualizations of the output format data

It is a work in progress. 

In [33]:
%gui qt5

In [1]:
%matplotlib inline

In [3]:
import sys
import starspace
import napari
import numpy as np
import itertools

/usr/local/lib/python3.7/site-packages/napari/__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.12.2. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [4]:
!{sys.executable} -m pip install -e ..

Obtaining file:///Users/ajc/projects/chanzuckerberg/spatial-warehouse
    100% |████████████████████████████████| 37.4MB 846kB/s eta 0:00:01    74% |████████████████████████        | 28.0MB 45.6MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 13.6MB/s ta 0:00:01


  Found existing installation: PyQt5-sip 4.19.15
    Uninstalling PyQt5-sip-4.19.15:
      Successfully uninstalled PyQt5-sip-4.19.15
  Found existing installation: PyQt5 5.12.1
    Uninstalling PyQt5-5.12.1:
      Successfully uninstalled PyQt5-5.12.1
  Found existing installation: starspace 0.0.1
    Uninstalling starspace-0.0.1:
      Successfully uninstalled starspace-0.0.1
  Running setup.py develop for starspace


## Load osmFISH

In [15]:
matrix = starspace.Matrix.load_zarr('../starspace/conversion_examples/osmfish-codeluppi-2018-nat-methods-somatosensory-cortex.matrix.zarr')

In [13]:
adata = matrix.to_anndata()

In [14]:
adata.write_h5ad('osmFISH_cortex.h5ad')

... storing 'biological_annotation' as categorical
... storing 'physical_annotation' as categorical
... storing 'channel' as categorical


## Look at points in napari

In [44]:
viewer = napari.Viewer()

In [17]:
spots = starspace.Spots.load_zarr('../starspace/conversion_examples/osmfish-codeluppi-2018-nat-methods-somatosensory-cortex.spots.zarr')

In [43]:
data = np.stack([-spots.y_spot, -spots.x_spot], axis=-1)

Napari isn't happy showing a quarter million points, but it can show 20k! Nick's gonna fix this.

In [45]:
gene_list = np.unique(spots['gene_name'])
base_colormaps = ['cyan', 'yellow', 'magenta', 'red', 'green', 'blue']
cycle_colors = itertools.cycle(base_colormaps)

# add the spots
for g, c in zip(gene_list, cycle_colors):
    print(g, c)
    inds = spots['gene_name'] == g
    # downsample 100x
    layer = viewer.add_points(data[inds][:100], face_color=c, edge_width=0, name=g)

Acta2 cyan
Aldoc yellow
Anln magenta
Apln red
Bmp4 green
Cnr1 blue
Cpne5 cyan
Crh yellow
Crhbp magenta
Ctps red
Flt1 green
Foxj1 blue
Gad2 cyan
Gfap yellow
Hexb magenta
Itpr2 red
Kcnip green
Klk6 blue
Lamp5 cyan
Lum yellow
Mfge8 magenta
Mrc1 red
Pdgfra green
Plp1 blue
Pthlh cyan
Rorb yellow
Serpinf1 magenta
Slc32a1 red
Sox10 green
Syt6 blue
Tbr1 cyan
Tmem6 yellow
Ttr magenta
Vip red
Vtn green


## Look at cell boundaries in napari

In [5]:
regions = starspace.Regions.load_zarr('../starspace/conversion_examples/osmfish-codeluppi-2018-nat-methods-somatosensory-cortex.regions.zarr/')

In [6]:
regions

<xarray.Regions 'array-baa123768d7ef2a6d2cdfa885e8f3eaa' (x_region: 31725, y_region: 51669)>
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int16)
Dimensions without coordinates: x_region, y_region
Attributes:
    assay:             osmFISH
    authors:           ['Simone Codeluppi', 'Lars E. Borm', 'Amit Zeisel', 'G...
    organism:          mouse
    publication_name:  Spatial organization of the somatosensory cortex revea...
    publication_url:   https://www.nature.com/articles/s41592-018-0175-z
    sample_type:       somatosensory cortex
    year:              2018

There's a bug here, the spots don't overlay properly (they're mirrored around x=y

In [48]:
viewer = napari.Viewer()
viewer.add_labels(regions)

/usr/local/lib/python3.7/site-packages/scipy/ndimage/interpolation.py:611: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


<Labels layer 'Labels' at 0x54b6ff160>